# Merge permno_cusip_ticker_permco and crsp_comp_bdx to get gvkey in it

In [38]:
import pandas as pd

In [39]:
# read the xlsx file
gvkey = pd.read_csv('/opt/hdd_1/research_hub/csr_project/Green_patent_dataset/original_dataset/CRSP_COMP_BDX_GVKEY.csv')
permco = pd.read_csv("/opt/hdd_1/research_hub/csr_project/Green_patent_dataset/original_dataset/PERMNO_CUSIP_TICKER_PERMCO.csv")

In [40]:
print("gvkey shape: ",gvkey.shape)
print("permco shape: ",permco.shape)

gvkey shape:  (11843, 6)
permco shape:  (187338, 18)


In [41]:
print("gvkey columns: ",gvkey.columns)
print("permco columns: ",permco.columns)

gvkey columns:  Index(['COMPANYID', 'PERMCO', 'GVKEY', 'SCORE', 'PREFERRED', 'DUPLICATE'], dtype='object')
permco columns:  Index(['PERMNO', 'FSYM_ID', 'FSYM_ID_KIND', 'PROPER_NAME', 'FSYM_REGIONAL_ID',
       'FSYM_SECURITY_ID', 'FS_PERM_SEC_ID', 'FACTSET_ENTITY_ID',
       'ENTITY_PROPER_NAME', 'CUSIP_FS', 'TICKER_EXCHANGE', 'PERMCO', 'CUSIP',
       'NCUSIP', 'TICKER', 'COMNAM', 'LINK_BDATE', 'LINK_EDATE'],
      dtype='object')


In [42]:
# Step 0: 篩選欄位 & 移除重複列
permco_less = permco[[
    'PERMNO', 'PROPER_NAME', 'ENTITY_PROPER_NAME', 'CUSIP_FS',
    'TICKER_EXCHANGE', 'PERMCO', 'CUSIP', 'NCUSIP', 'TICKER',
    'COMNAM', 'LINK_BDATE', 'LINK_EDATE'
]].copy()

permco_less.drop_duplicates(inplace=True)

# Step 1: 確保 LINK_EDATE 是數值型別（避免 NaN 比較出錯）
permco_less['LINK_EDATE'] = pd.to_numeric(permco_less['LINK_EDATE'], errors='coerce')

# Step 2: 根據 PERMNO 保留 LINK_EDATE 最大的那一筆
latest_permco = permco_less.loc[
    permco_less.groupby('PERMNO')['LINK_EDATE'].idxmax()
].reset_index(drop=True)

In [44]:
# 確保是字串格式
latest_permco['TICKER_EXCHANGE'] = latest_permco['TICKER_EXCHANGE'].astype(str)

# 拆分欄位
latest_permco[['TICKER_SYMBOL', 'EXCHANGE']] = latest_permco['TICKER_EXCHANGE'].str.split('-', expand=True)
latest_permco = latest_permco[latest_permco['EXCHANGE'].isin(['NAS', 'NYS'])].reset_index(drop=True)
latest_permco.drop(columns=['TICKER_EXCHANGE', 'TICKER_SYMBOL', 'LINK_BDATE',	'LINK_EDATE'], inplace=True)
latest_permco

,PERMNO,PROPER_NAME,ENTITY_PROPER_NAME,CUSIP_FS,PERMCO,CUSIP,NCUSIP,TICKER,COMNAM,EXCHANGE
0,10002,BancTrust Financial Group Inc.,"BancTrust Financial Group, Inc.",05978R107,7954,05978R10,05978R10,BTFG,BANCTRUST FINANCIAL GROUP INC,NAS
1,10003,Great Ctry Bank Ansonia Ct,NaN,390318103,7957,39031810,39031810,GCBK,GREAT COUNTRY BK ASONIA CT,NAS
2,10006,Acf Industries,NaN,000800102,22156,00080010,00080010,ACF,A C F INDUSTRIES INC,NYS
3,10009,Iroquois Bancorp Inc.,"Iroquois Bancorp, Inc. (New York)",463347104,7965,46334710,46334710,IROQ,IROQUOIS BANCORP INC,NAS
4,10010,Cabot Medical Corporation,Cabot Medical Corp.,127095107,7967,12709510,12709510,CBOT,CABOT MEDICAL CORP,NAS
...,...,...,...,...,...,...,...,...,...,...
16293,93428,"BroadSoft, Inc.","BroadSoft, Inc.",11133B409,53446,11133B40,11133B40,BSFT,BROADSOFT INC,NAS
16294,93429,Cboe Global Markets Inc,"Cboe Global Markets, Inc.",12503M108,53447,12503M10,12503M10,CBOE,C B O E GLOBAL MARKETS INC,NAS
16295,93431,Pactera Technology International Ltd. Sponsore...,Pactera Technology International Ltd.,695255109,53449,69525510,69525510,PACT,PACTERA TECHNOLOGY INTL LTD,NAS
16296,93434,S&W Seed Company,S&W Seed Co.,785135104,53427,78513510,78513510,SANW,S & W SEED CO,NAS


In [45]:
permco_gvkey = pd.merge(latest_permco, gvkey[['COMPANYID', 'PERMCO', 'GVKEY']], how='left', on='PERMCO')
print("permco_gvkey shape: ",permco_gvkey.shape)

permco_gvkey shape:  (16722, 12)


In [46]:
permco_gvkey_without_na = permco_gvkey.dropna(subset=['GVKEY'])
print("permco_gvkey_without_na shape: ",permco_gvkey_without_na.shape)

permco_gvkey_without_na shape:  (7669, 12)


In [47]:
permco_gvkey_without_na.to_csv('/opt/hdd_1/research_hub/csr_project/Green_patent_dataset/original_dataset/permno_cusip_ticker_permco_gvkey_without_na.csv', index=False)

## Merge green patent count and forward citation

In [ ]:
import pandas as pd

In [ ]:
forward = pd.read_csv("/opt/hdd_1/research_hub/csr_project/Green_patent_dataset/original_dataset/total_5yr_forward_citations_current.csv")
count = pd.read_csv("/opt/hdd_1/research_hub/csr_project/Green_patent_dataset/original_dataset/green_patents_count.csv")

In [ ]:
print("forward.columns: ",forward.columns)
print("count.columns: ",count.columns)

forward.columns:  Index(['gvkey_numeric', 'filing_year', 'total_5yr_forward_citations',
       'within_5_years'],
      dtype='object')
count.columns:  Index(['gvkey_numeric', 'filing_year', 'patents_count'], dtype='object')


In [ ]:
forward_count = pd.merge(forward, count, how='left', on=['gvkey_numeric', 'filing_year'])
forward_count.to_csv('/opt/hdd_1/research_hub/csr_project/Green_patent_dataset/original_dataset/total_5yr_forward_citations_v2_green_current_count.csv', index=False)

## Merge to get ticker

In [ ]:
import pandas as pd

In [ ]:
forward_count = pd.read_csv("/opt/hdd_1/research_hub/csr_project/Green_patent_dataset/merged_dataset/total_5yr_forward_citations_current_count.csv")
mapping = pd.read_csv("/opt/hdd_1/research_hub/csr_project/Green_patent_dataset/merged_dataset/permno_cusip_ticker_permco_gvkey_without_na.csv")

In [ ]:
print(forward_count.columns)
print(mapping.columns)

Index(['gvkey_numeric', 'filing_year', 'total_5yr_forward_citations',
       'within_5_years', 'patents_count'],
      dtype='object')
Index(['PERMNO', 'FSYM_ID', 'FSYM_ID_KIND', 'PROPER_NAME', 'FSYM_REGIONAL_ID',
       'FSYM_SECURITY_ID', 'FS_PERM_SEC_ID', 'FACTSET_ENTITY_ID',
       'ENTITY_PROPER_NAME', 'CUSIP_FS', 'TICKER_EXCHANGE', 'PERMCO', 'CUSIP',
       'NCUSIP', 'TICKER', 'COMNAM', 'LINK_BDATE', 'LINK_EDATE', 'COMPANYID',
       'GVKEY'],
      dtype='object')


In [ ]:
forward_count_ticker = pd.merge(forward_count, mapping[['GVKEY', 'PERMNO', 'CUSIP', 'TICKER']], how='left', left_on='gvkey_numeric', right_on='GVKEY')

In [ ]:
forward_count_ticker_dropna = forward_count_ticker.dropna()
forward_count_ticker_dropna.to_csv("/opt/hdd_1/research_hub/csr_project/Green_patent_dataset/merged_dataset/total_5yr_forward_citations_current_count_ticker.csv", index=False)

In [ ]:
report_ticker = pd.read_csv("/opt/hdd_1/research_hub/csr_project/Green_patent_dataset/company_name/company_name_ticker.csv")

In [ ]:
forward_count_ticker_dropna = forward_count_ticker_dropna[forward_count_ticker_dropna['TICKER'].isin(report_ticker['ticker'].values)]
forward_count_ticker_dropna.to_csv("/opt/hdd_1/research_hub/csr_project/Green_patent_dataset/merged_dataset/total_5yr_forward_citations_current_count_ticker_within_report.csv", index=False)

# Merge the ticker in mapping to gp_values by permno

In [1]:
import pandas as pd

In [ ]:
gp_value = pd.read_csv("/opt/hdd_1/research_hub/csr_project/Green_patent_dataset/original_dataset/green_patents_values_filing.csv")
mapping = pd.read_csv("/opt/hdd_1/research_hub/csr_project/Green_patent_dataset/original_dataset/PERMNO_CUSIP_TICKER_PERMCO.csv")
csr_ticker_name = pd.read_csv("/opt/hdd_1/research_hub/csr_project/CSR Reporting/csr_ticker_name.csv")

In [68]:
from difflib import SequenceMatcher
import pandas as pd

def merge_and_match_best_company(gp_value, mapping, csr_ticker_name):
    """
    將 gp_value 根據 mapping 對應出 TICKER 與 COMNAM，
    再根據 TICKER 對應到 csr_ticker_name 的 COMPANY，
    並保留每筆 (permno, year) 最相似的 COMPANY 一筆。

    Parameters:
        gp_value (DataFrame): 包含 permno, year 等欄位
        mapping (DataFrame): 包含 PERMNO, TICKER, COMNAM
        csr_ticker_name (DataFrame): 包含 TICKER, COMPANY

    Returns:
        DataFrame: 合併後只保留最相似 COMPANY 的資料
    """
    
    # Step 1: 合併 gp_value 與 mapping（PERMNO -> TICKER, COMNAM）
    merged = pd.merge(
        gp_value, 
        mapping[['PERMNO', 'TICKER', 'COMNAM']], 
        how='left', 
        left_on='permno', 
        right_on='PERMNO'
    )
    merged['year'] = merged['year'].astype('int')

    # Step 2: 合併 TICKER -> COMPANY
    merged = pd.merge(
        merged, 
        csr_ticker_name[['ticker', 'COMPANY']], 
        how='left', 
        left_on='TICKER',
        right_on='ticker'
    )

    # Step 3: 對每組 permno-year 找出最相似的 COMPANY（比 COMNAM）
    def get_best_match(group):
        if group.shape[0] == 1:
            return group
        comnam = str(group['COMNAM'].iloc[0])
        group['similarity'] = group['COMPANY'].apply(lambda x: SequenceMatcher(None, comnam, str(x)).ratio())
        return group.loc[[group['similarity'].idxmax()]]

    best_match = merged.groupby(['permno', 'year'], group_keys=False).apply(get_best_match)

    # Step 4: 移除 similarity 欄（如不需要）
    best_match = best_match.drop(columns=['similarity'], errors='ignore')

    return best_match


In [69]:
best_match = merge_and_match_best_company(gp_value, mapping, csr_ticker_name)

/tmp/ipykernel_989966/3165612530.py:46: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  best_match = merged.groupby(['permno', 'year'], group_keys=False).apply(get_best_match)


In [ ]:
best_match.to_csv("/opt/hdd_1/research_hub/csr_project/Green_patent_dataset/merged_dataset/green_patent_values_best_match.csv", index=False)

In [16]:
import pandas as pd
best_match = pd.read_csv("/opt/hdd_1/research_hub/csr_project/Green_patent_dataset/merged_dataset/green_patent_values_best_match.csv")
report_ticker = pd.read_csv("/opt/hdd_1/research_hub/csr_project/Green_patent_dataset/company_name/company_name_ticker.csv")

In [17]:
best_match = best_match[best_match['TICKER'].isin(report_ticker['ticker'].values)]
best_match.to_csv("/opt/hdd_1/research_hub/csr_project/Green_patent_dataset/merged_dataset/green_patent_values_best_match_within_report.csv", index=False)

# Delete report with year2 (overlapping)

In [48]:
import pandas as pd

In [49]:
report_all = pd.read_csv("/opt/hdd_1/research_hub/csr_project/CSR Reporting/Dataset/report_all.csv")

In [50]:
report_all_without_overlap = report_all[report_all['year2'].isna()]
report_all_without_overlap.drop(columns=['year2'], inplace=True)
report_all_without_overlap.to_csv("/opt/hdd_1/research_hub/csr_project/CSR Reporting/Dataset/report_all_without_overlap.csv", index=False)

/tmp/ipykernel_1028577/2918103659.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  report_all_without_overlap.drop(columns=['year2'], inplace=True)


# Merge report and patent

## Merge report and forward, count

In [53]:
import pandas as pd
import numpy as np

In [54]:
report = pd.read_csv("/opt/hdd_1/research_hub/csr_project/CSR Reporting/Dataset/report_company_without_overlap.csv")
forward_count_ticker = pd.read_csv("/opt/hdd_1/research_hub/csr_project/Green_patent_dataset/merged_dataset/total_5yr_forward_citations_current_count_ticker.csv")

In [55]:
report['ticker'] = report['ticker'].str.upper()
report

,folder_name,exchange_x,ticker,year,company_name,exchange_y
0,NASDAQ_AAL_2007,NASDAQ,AAL,2007,American Airlines Group,NAS
1,NASDAQ_AAL_2008,NASDAQ,AAL,2008,American Airlines Group,NAS
2,NASDAQ_AAL_2009,NASDAQ,AAL,2009,American Airlines Group,NAS
3,NASDAQ_AAL_2011,NASDAQ,AAL,2011,American Airlines Group,NAS
4,NASDAQ_AAL_2012,NASDAQ,AAL,2012,American Airlines Group,NAS
...,...,...,...,...,...,...
6053,NYSE_hpq_2017,NYSE,HPQ,2017,HP Inc,NYS
6054,NYSE_hpq_2018,NYSE,HPQ,2018,HP Inc,NYS
6055,NYSE_hpq_2019,NYSE,HPQ,2019,HP Inc,NYS
6056,NYSE_hpq_2020,NYSE,HPQ,2020,HP Inc,NYS


In [56]:
forward_count_ticker = forward_count_ticker.drop_duplicates()
forward_count_ticker['TICKER'] = forward_count_ticker['TICKER'].str.upper()
forward_count_ticker

,gvkey_numeric,filing_year,total_5yr_forward_citations,within_5_years,patents_count,GVKEY,PERMNO,CUSIP,TICKER
0,4839,2004,0,0.0,3,4839.0,25785.0,34537086,F
1,4839,2005,0,0.0,6,4839.0,25785.0,34537086,F
2,4839,2006,4,2.0,5,4839.0,25785.0,34537086,F
3,4839,2007,280,24.0,28,4839.0,25785.0,34537086,F
4,4839,2008,358,27.0,34,4839.0,25785.0,34537086,F
...,...,...,...,...,...,...,...,...,...
8760,23051,2014,3,1.0,1,23051.0,76393.0,60474510,MIRR
8761,65944,2011,0,0.0,1,65944.0,85686.0,04546C20,DPMD
8762,65944,2011,0,0.0,1,65944.0,85686.0,04546C20,DMI
8763,65944,2011,0,0.0,1,65944.0,85686.0,04546C20,DEPO


In [57]:
# 標準化 ticker 為大寫（確保一致）
report['ticker'] = report['ticker'].str.upper()
forward_count_ticker['TICKER'] = forward_count_ticker['TICKER'].str.upper()

# 取交集
common_tickers = set(report['ticker']).intersection(set(forward_count_ticker['TICKER']))

# 轉成 DataFrame 方便查看
common_ticker_df = pd.DataFrame(sorted(common_tickers), columns=['TICKER'])

# 顯示結果
print(f"✅ 共有 {len(common_ticker_df)} 個 ticker 同時出現在 report 和 forward_count_ticker 中")
print(common_ticker_df.head())


✅ 共有 392 個 ticker 同時出現在 report 和 forward_count_ticker 中
  TICKER
0      A
1     AA
2   AAPL
3   ABBV
4    ABT


In [58]:
# Step 1: 先合併（與原始邏輯一致）
report_forward_count = pd.merge(
    report,
    forward_count_ticker[['TICKER', 'filing_year', 'total_5yr_forward_citations', 'within_5_years', 'patents_count']],
    how='left',
    left_on=['ticker', 'year'],  # 根據你的欄位名稱應該是 year1
    right_on=['TICKER', 'filing_year']
)

# Step 2: 建立一個 ticker 的交集集合
common_tickers = set(report['ticker']).intersection(set(forward_count_ticker['TICKER']))

# Step 3: 建立條件：在交集內的 ticker，但數值為 NaN（表示年份沒對上）
mask = (
    report_forward_count['ticker'].isin(common_tickers) &
    report_forward_count['total_5yr_forward_citations'].isna()
)

# Step 4: 對這些條件補上 0
cols_to_fill = ['total_5yr_forward_citations', 'within_5_years', 'patents_count']
report_forward_count.loc[mask, cols_to_fill] = 0

# Step 5: 此時只補了你要補的，其他 NaN 不動，接著轉型時只針對非 NaN 行
for col in cols_to_fill:
    report_forward_count[col] = report_forward_count[col].apply(lambda x: int(x) if pd.notna(x) else np.nan)

print("✅ 已補上 forward 缺年份的 0，其他保持 NaN")

✅ 已補上 forward 缺年份的 0，其他保持 NaN


In [59]:
report_forward_count

,folder_name,exchange_x,ticker,year,company_name,exchange_y,TICKER,filing_year,total_5yr_forward_citations,within_5_years,patents_count
0,NASDAQ_AAL_2007,NASDAQ,AAL,2007,American Airlines Group,NAS,NaN,NaN,NaN,NaN,NaN
1,NASDAQ_AAL_2008,NASDAQ,AAL,2008,American Airlines Group,NAS,NaN,NaN,NaN,NaN,NaN
2,NASDAQ_AAL_2009,NASDAQ,AAL,2009,American Airlines Group,NAS,NaN,NaN,NaN,NaN,NaN
3,NASDAQ_AAL_2011,NASDAQ,AAL,2011,American Airlines Group,NAS,NaN,NaN,NaN,NaN,NaN
4,NASDAQ_AAL_2012,NASDAQ,AAL,2012,American Airlines Group,NAS,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
6105,NYSE_hpq_2017,NYSE,HPQ,2017,HP Inc,NYS,NaN,NaN,NaN,NaN,NaN
6106,NYSE_hpq_2018,NYSE,HPQ,2018,HP Inc,NYS,NaN,NaN,NaN,NaN,NaN
6107,NYSE_hpq_2019,NYSE,HPQ,2019,HP Inc,NYS,NaN,NaN,NaN,NaN,NaN
6108,NYSE_hpq_2020,NYSE,HPQ,2020,HP Inc,NYS,NaN,NaN,NaN,NaN,NaN


In [60]:
# Step 1: 合併 report 和 forward 資料
report_forward_count_status = pd.merge(
    report,
    forward_count_ticker[['TICKER', 'filing_year', 'total_5yr_forward_citations', 'within_5_years', 'patents_count']],
    how='left',
    left_on=['ticker', 'year'],
    right_on=['TICKER', 'filing_year']
)

# Step 2: 定義 ticker 的交集
common_tickers = set(report['ticker']).intersection(set(forward_count_ticker['TICKER']))

# Step 3: 建立 match_status 欄位
def classify_match(row):
    if pd.notna(row['total_5yr_forward_citations']):
        return 'full_match'
    elif row['ticker'] in common_tickers:
        return 'missing_year'
    else:
        return 'ticker_not_found'

report_forward_count_status['match_status'] = report_forward_count_status.apply(classify_match, axis=1)

# Step 4: 僅對 missing_year 的資料補 0
cols_to_fill = ['total_5yr_forward_citations', 'within_5_years', 'patents_count']
report_forward_count_status.loc[report_forward_count_status['match_status'] == 'missing_year', cols_to_fill] = 0

# Step 5: 將補過的值轉 int，其餘仍為 NaN
for col in cols_to_fill:
    report_forward_count_status[col] = report_forward_count_status[col].apply(lambda x: int(x) if pd.notna(x) else np.nan)

# Done!
print("✅ match_status 欄位建立完成，forward 資訊依狀況正確補值")
print(report_forward_count_status[['ticker', 'year', 'match_status']].value_counts())


✅ match_status 欄位建立完成，forward 資訊依狀況正確補值
ticker  year  match_status    
APTV    2019  missing_year        5
        2018  full_match          5
        2021  missing_year        5
        2022  missing_year        5
        2020  missing_year        5
                                 ..
EXR     2020  ticker_not_found    1
        2019  ticker_not_found    1
EXPD    2021  ticker_not_found    1
        2020  ticker_not_found    1
F       2018  full_match          1
Name: count, Length: 5909, dtype: int64


In [63]:
# print ticker_not_found
ticker_not_found = report_forward_count_status[report_forward_count_status['match_status'] == 'ticker_not_found']
ticker_not_found

,folder_name,exchange_x,ticker,year,company_name,exchange_y,TICKER,filing_year,total_5yr_forward_citations,within_5_years,patents_count,match_status
0,NASDAQ_AAL_2007,NASDAQ,AAL,2007,American Airlines Group,NAS,NaN,NaN,NaN,NaN,NaN,ticker_not_found
1,NASDAQ_AAL_2008,NASDAQ,AAL,2008,American Airlines Group,NAS,NaN,NaN,NaN,NaN,NaN,ticker_not_found
2,NASDAQ_AAL_2009,NASDAQ,AAL,2009,American Airlines Group,NAS,NaN,NaN,NaN,NaN,NaN,ticker_not_found
3,NASDAQ_AAL_2011,NASDAQ,AAL,2011,American Airlines Group,NAS,NaN,NaN,NaN,NaN,NaN,ticker_not_found
4,NASDAQ_AAL_2012,NASDAQ,AAL,2012,American Airlines Group,NAS,NaN,NaN,NaN,NaN,NaN,ticker_not_found
...,...,...,...,...,...,...,...,...,...,...,...,...
6105,NYSE_hpq_2017,NYSE,HPQ,2017,HP Inc,NYS,NaN,NaN,NaN,NaN,NaN,ticker_not_found
6106,NYSE_hpq_2018,NYSE,HPQ,2018,HP Inc,NYS,NaN,NaN,NaN,NaN,NaN,ticker_not_found
6107,NYSE_hpq_2019,NYSE,HPQ,2019,HP Inc,NYS,NaN,NaN,NaN,NaN,NaN,ticker_not_found
6108,NYSE_hpq_2020,NYSE,HPQ,2020,HP Inc,NYS,NaN,NaN,NaN,NaN,NaN,ticker_not_found


In [64]:
report_forward_count_dropna = report_forward_count.dropna(subset=['total_5yr_forward_citations'])
report_forward_count_dropna.drop(columns=['filing_year', 'exchange_y', 'TICKER'], inplace=True)

/tmp/ipykernel_1038622/898161925.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  report_forward_count_dropna.drop(columns=['filing_year', 'exchange_y', 'TICKER'], inplace=True)


In [66]:
report_forward_count_dropna.to_csv("/opt/hdd_1/research_hub/csr_project/CSR Reporting/Dataset/report_forward_count.csv", index=False)

## Count how many count company with report

In [6]:
import pandas as pd

In [7]:
forward_count_within = pd.read_csv("/opt/hdd_1/research_hub/csr_project/Green_patent_dataset/merged_dataset/total_5yr_forward_citations_current_count_ticker_within_report.csv")

In [55]:
len(forward_count_within['TICKER'].unique())

602

## forward_count_ticker 中有哪些 TICKER ，在 report 中沒有出現過

In [10]:
import pandas as pd

In [13]:
report_all = pd.read_csv("/opt/hdd_1/research_hub/csr_project/CSR Reporting/Dataset/report_all.csv")
forward_count_ticker = pd.read_csv("/opt/hdd_1/research_hub/csr_project/Green_patent_dataset/merged_dataset/total_5yr_forward_citations_current_count_ticker.csv")

In [15]:
forward_count_ticker['TICKER'].unique()

array(['F', 'QCOM', 'RHR', ..., 'DMI', 'DEPO', 'FITB'],
      shape=(2071,), dtype=object)

In [16]:
report_all['ticker'] = report_all['ticker'].str.upper()
report_all['ticker'].unique()

array(['AAL', 'AAON', 'AAPL', ..., 'ZMH', 'ZNH', 'HPQ'],
      shape=(1134,), dtype=object)

In [17]:
# 標準化 ticker 為大寫（確保一致）
report_all['ticker'] = report_all['ticker'].str.upper()
forward_count_ticker['TICKER'] = forward_count_ticker['TICKER'].str.upper()

# 取得各自唯一的 ticker 集合
report_all_tickers = set(report_all['ticker'].unique())
forward_tickers = set(forward_count_ticker['TICKER'].unique())

# 找出只存在於 forward_count_ticker 中的 ticker
missing_tickers = forward_tickers - report_all_tickers

# 轉成 DataFrame 顯示
missing_ticker_df = pd.DataFrame(sorted(missing_tickers), columns=['TICKER'])

# 顯示結果
print(f"✅ 有 {len(missing_ticker_df)} 個 ticker 僅出現在 forward_count_ticker 中")
print(missing_ticker_df.head())


✅ 有 1667 個 ticker 僅出現在 forward_count_ticker 中
  TICKER
0   AABA
1   AAII
2   AAOI
3    AAS
4   AATI


In [ ]:
# # Step 1: 把 report 的 ticker 標準化成大寫
# report['ticker'] = report['ticker'].str.upper()

# # Step 2: 只取出報告中有的 (ticker, year)
# report_pairs = report[['ticker', 'year']].drop_duplicates().rename(
#     columns={'ticker': 'TICKER', 'year': 'filing_year'}
# )

# # Step 3: 用 outer merge 加上 indicator 來判斷來源
# merged = forward_count_ticker[['TICKER', 'filing_year']].drop_duplicates().merge(
#     report_pairs,
#     on=['TICKER', 'filing_year'],
#     how='left',
#     indicator=True
# )

# # Step 4: 篩選 _merge == 'left_only'，代表只有在 forward_count_ticker 中出現
# missing_in_report = merged[merged['_merge'] == 'left_only'][['TICKER', 'filing_year']].reset_index(drop=True)

# # 顯示結果
# print(f"✅ 有 {len(missing_in_report)} 筆 TICKER + year 組合在 forward_count_ticker 中但不在 report 中")
# print(missing_in_report.head())
# missing_in_report

✅ 有 7786 筆 TICKER + year 組合在 forward_count_ticker 中但不在 report 中
  TICKER  filing_year
0      F         2004
1      F         2005
2      F         2006
3      F         2007
4      F         2008


,TICKER,filing_year
0,F,2004
1,F,2005
2,F,2006
3,F,2007
4,F,2008
...,...,...
7781,MIRR,2014
7782,DPMD,2011
7783,DMI,2011
7784,DEPO,2011
